In [5]:
import requests
import json
from datetime import datetime, timedelta

api_url = "https://api-web.nhle.com"

with open('teams.json', 'r') as file:
    teams = json.load(file)

# Get scores for all games in 2022-2023 
start_date = datetime(2022, 10, 7)  
end_date = datetime(2023, 4, 15)     

schedule = []

current_date = start_date
while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    
    response = requests.get(f"{api_url}/v1/score/{formatted_date}")
    
    if response.status_code == 200:
        scores_data = response.json()
        schedule.append(scores_data)
    else:
        print(f"Failed to fetch scores for {formatted_date}. Status code: {response.status_code}")
    
    current_date += timedelta(days=1)

In [6]:
print(schedule[20]['games'][0])

{'id': 2022020111, 'season': 20222023, 'gameType': 2, 'gameDate': '2022-10-27', 'venue': {'default': 'TD Garden'}, 'startTimeUTC': '2022-10-27T23:00:00Z', 'easternUTCOffset': '-04:00', 'venueUTCOffset': '-04:00', 'tvBroadcasts': [{'id': 31, 'market': 'H', 'countryCode': 'US', 'network': 'NESN', 'sequenceNumber': 1}, {'id': 287, 'market': 'N', 'countryCode': 'CA', 'network': 'SNE', 'sequenceNumber': 1}, {'id': 288, 'market': 'N', 'countryCode': 'CA', 'network': 'SNO', 'sequenceNumber': 1}, {'id': 290, 'market': 'N', 'countryCode': 'CA', 'network': 'SNP', 'sequenceNumber': 1}, {'id': 351, 'market': 'A', 'countryCode': 'US', 'network': 'BSDET', 'sequenceNumber': 1}], 'gameState': 'OFF', 'gameScheduleState': 'OK', 'awayTeam': {'id': 17, 'name': {'default': 'Red Wings'}, 'abbrev': 'DET', 'score': 1, 'sog': 29, 'logo': 'https://assets.nhle.com/logos/nhl/svg/DET_light.svg'}, 'homeTeam': {'id': 6, 'name': {'default': 'Bruins'}, 'abbrev': 'BOS', 'score': 5, 'sog': 33, 'logo': 'https://assets.nh

In [7]:
# Parse game data
total_games = 0
scores = []
for day in schedule:
    total_games += len(day['games'])
    for game in day['games']:
        awayTeam = game['awayTeam']['abbrev']
        awayScore = game['awayTeam']['score']
        homeTeam = game['homeTeam']['abbrev']
        homeScore = game['homeTeam']['score']
        homeWin = homeScore > awayScore

        score_data = {
            "awayTeam": awayTeam,
            "awayScore": awayScore,
            "homeTeam": homeTeam,
            "homeScore": homeScore,
            "homeWin": homeWin
        }
        
        scores.append(score_data)


In [8]:
import csv 

fields = ['awayTeam', 'awayScore', 'homeTeam', 'homeScore', 'homeWin']

with open('all_scores.csv', 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fields)
    
    writer.writeheader()
    
    for score in scores:
        writer.writerow(score)

print("CSV file 'all_scores.csv' has been created.")


CSV file 'all_scores.csv' has been created.


In [41]:
# Get club stats for each team in 2022-2023
import json
with open('teams.json', 'r') as file:
    teams = json.load(file)

# List of player ids for each team
team_rosters = {}

# Cumulative team stats for each team
team_stats = {}
for team in teams:
    abbr = team['abbreviation']
    club_stats = requests.get(f"{api_url}/v1/club-stats/{abbr}/20222023/2").json()

    team_rosters[abbr] = []
    team_stats[abbr] = {
        'goals_per_game': 0,
        'netPlusMinus': 0,
        'totalPIM': 0,
        'PPG': 0,
        'SHG': 0,
        'shots_per_game': 0,
        'goals_against_per_game': 0,
        'shots_against_per_game': 0
    }
    for skater in club_stats['skaters']:
        id = skater['playerId']
        team_rosters[abbr].append(id)

        stats = team_stats[abbr]
        stats['goals_per_game'] += skater['goals']
        stats['netPlusMinus'] += skater['plusMinus']
        stats['totalPIM'] += skater['penaltyMinutes']
        stats['PPG'] += skater['powerPlayGoals']
        stats['SHG'] += skater['shorthandedGoals']
        stats['shots_per_game'] += skater['shots']

    for goalie in club_stats['goalies']:                
        stats = team_stats[abbr]
        stats['shots_against_per_game'] += goalie['shotsAgainst']
        stats['goals_against_per_game'] += goalie['goalsAgainst']
    
        
    team_stats[abbr]['goals_per_game'] /= 82
    team_stats[abbr]['shots_per_game'] /= 82
    team_stats[abbr]['goals_against_per_game'] /= 82
    team_stats[abbr]['shots_against_per_game'] /= 82

In [26]:
output = 'rosters.json'

# Export team_rosters to a JSON file
with open(output, 'w') as file:
    json.dump(team_rosters, file)

print(f"Team rosters have been exported to {output}.")

Team rosters have been exported to rosters.json.


In [42]:
output = 'team_stats.json'

# Export team_rosters to a JSON file
with open(output, 'w') as file:
    json.dump(team_stats, file)

print(f"Team stats have been exported to {output}.")

Team stats have been exported to team_stats.json.
